# Problem 1. IV in Practice.

In [1]:
#import required packages
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy as sc
import matplotlib as plt
import sys 
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
#set notebook parameters to show data in pandas frames
pd.options.display.max_columns = 12
pd.options.display.max_rows = 5

#establish random seed
np.random.seed(1)

In [3]:
#grab the dataframe
df=pd.read_csv("lottery_newstata.csv")
df

,female,year,lotcateg,z,d,lnw
0,0,1988,4,0,1,2.751068
1,1,1989,5,1,1,4.365279
...,...,...,...,...,...,...
1474,1,1989,5,1,1,2.857421
1475,0,1989,6,0,0,2.603315


# 1. Use an  $F-$ test to explore if the intrument is relevant.

In [4]:
#Endogenous is Y
endog = df["lnw"]

#Exog is a constant and Z
exog = sm.add_constant(df["z"])

#We now can fit regression
model = sm.OLS(endog, exog).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    lnw   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     13.73
Date:                Fri, 04 Dec 2020   Prob (F-statistic):           0.000219
Time:                        21:20:39   Log-Likelihood:                -967.54
No. Observations:                1476   AIC:                             1939.
Df Residuals:                    1474   BIC:                             1950.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.0873      0.022    141.359      0.000       3.045       3.130
z              0.0974      0.026      3.706      0.000       0.046       0.149
==============================================================================
Omnibus:                       55.515   Durbin-Watson:                   1.943
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              116.071
Skew:                           0.232   Prob(JB):                     6.24e-26
Kurtosis:                       4.293   Cond. No.                         3.36
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
model.f_test("z")

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[13.73199162]]), p=0.00021862984453511748, df_denom=1.47e+03, df_num=1>

## <span style="color:red">We make conclusion that with p=0.0002 the instrument is relevant</span>

# 2. Estimate and proviode inference for the casual effect of graduation on post-graduation log wage, justifying your clustering. 

## <span style="color:green"> I did not quite understand the setup of the problem. An individual cannot "GO TO SCHOOL" if he is not admitted, because Admissions are decided by the lottery. So, we should not have it in the dataset that Z==0 but D==1. Was the intention to state "Individuals decide/self-select to go to school based on binary D, yet lottery Z defines if the individual actually was admited". It is also not clear whether each individual who was admitted actually graduated???? </span>
 

## <span style="color:green"> Ok, assume D==1 stands for submitting admission application, Z==1 tands for randomly selected for admission, then D==1,Z==0 makes sense. But then, how do we have D==0, and Z==1? Does it mean that someone didn't actually apply for admission, but then was forced to get schooling by all-mighty government and each student of these graduated? <br><br> I am trying to solve it as is ... </span>

## I am making assumption of the following process:
1. Individual applies to go to school D==1
2. Individual is admitted according to the lottery Z and graduates. We assume no dropouts. Z means random admission and graduation. 

### <span style="color:blue"> Clustering will be done by year / category. Because the lottery is exogenous at this aggregation. </span>

In [6]:
#get list of categories
categs=df["lotcateg"].unique()
categs

array([4, 5, 6, 3])

In [7]:
#get list of years
years = df["year"].unique()
years

array([1988, 1989])

In [8]:
from linearmodels.iv import IV2SLS

#create a new dataframe where will sum the results
summary =pd.DataFrame(index=["Instrument Beta","Casual Effect", "t-stat of Beta", "Category", "Year"])

#iterate/#loop over the list, run regression, extract beta value for Z and t-statistic, append to the dataframe 
for year in years:
    for cat in categs:

        #fit model for every cat
        mod = IV2SLS.from_formula('lnw ~ 1  + [d ~ z]', df[df["lotcateg"]==cat][df["year"]==year]).fit()

        #extract params
        z=mod.params.d
        c = np.exp(z)
        t_stat = mod.tstats.d

        #shove them all into one list
        add = [z,c,t_stat,str(cat),str(year)]

        #get new column name in dataset to append the result
        col = "lotcateg_" + str(cat) + "_" + str(year)

        #now transpose and add the list as a column
        summary[col]= add
    
    


In [9]:
#enjoy the results -we will use this table in #3 at the end. 
summary

,lotcateg_4_1988,lotcateg_5_1988,lotcateg_6_1988,lotcateg_3_1988,lotcateg_4_1989,lotcateg_5_1989,lotcateg_6_1989,lotcateg_3_1989
Instrument Beta,0.119505,0.159243,0.209577,-0.803662,0.0119385,0.518693,0.145346,1.56419
Casual Effect,1.12694,1.17262,1.23316,0.447686,1.01201,1.67983,1.15644,4.77881
t-stat of Beta,0.546878,1.60704,2.35277,-0.683861,0.0505322,2.34764,1.60404,1.55242
Category,4,5,6,3,4,5,6,3
Year,1988,1988,1988,1988,1989,1989,1989,1989


### <span style="color:blue"> We notice that on a cluster level Instrument varies, as does its statistical significance. </span>  <br>
### <span style="color:blue"> Interpretation of infecence: Casual Effect is multiplicative value of a wage of those who graduated comparing with those who did not. It is obtaned by taking exponent of Beta coefficient of Z. </span>

# 3. Verify that the instrumental variable and two-stage least squares estimators provide the same estimate of the causal eﬀect in 2.

### <span style="color:blue"> To accomplish this task, we (first) need to predict $\hat{D}$ based on Z - by virtue of regressing D on Z, then we obtain clustered betas for $\hat{D}$ by regressing $\hat{D}$  on logwage. We aim to prove that those clustered betas are the same as clustered betas from #2</span>

In [10]:
#1. Obtain clustered betas and alphas for each of the category clusters. 
#create a new dataframe where will sum the results
summary1 =pd.DataFrame(index=["Alhpa_of_D","Beta of D", "Category", "Year"])

#iterate/#loop over the list, run regression, extract beta value for Z and t-statistic, append to the dataframe 

for year in years: 
    for cat in categs:

        #fit model for every cat
        endog = df["d"][df["lotcateg"]==cat][df["year"]==year]
        exog = sm.add_constant(df["z"][df["lotcateg"]==cat][df["year"]==year])

        #fit model for every cat
        mod = sm.OLS(endog, exog).fit()

        #extract params
        d=mod.params.z
        c= mod.params.const


        #shove them all into one list
        add = [c,d, str(cat), str(year)]

        #get new column name in dataset to append the result
        col = "lotcateg_" + str(cat) + "_" + str(year)

        #now transpose and add the list as a column
        summary1[col]= add
summary1

,lotcateg_4_1988,lotcateg_5_1988,lotcateg_6_1988,lotcateg_3_1988,lotcateg_4_1989,lotcateg_5_1989,lotcateg_6_1989,lotcateg_3_1989
Alhpa_of_D,0.56,0.346667,0.275862,0.777778,0.675676,0.61194,0.257732,0.6
Beta of D,0.339371,0.601159,0.657471,0.168889,0.272376,0.325123,0.668935,0.305405
Category,4,5,6,3,4,5,6,3
Year,1988,1988,1988,1988,1989,1989,1989,1989


In [11]:
#create a function which creates a vector of alphas depending on category
def get_alpha(column):
    if column == "4_1988":
        alpha=summary1.iloc[0,0]
    elif column == "5_1988":
        alpha=summary1.iloc[0,1]
    elif column == "6_1988":
        alpha=summary1.iloc[0,2]
    elif column == "3_1988":
        alpha=summary1.iloc[0,3]      
    elif column == "4_1989":
        alpha=summary1.iloc[0,4]
    elif column == "5_1989":
        alpha=summary1.iloc[0,5]
    elif column == "6_1989":
        alpha=summary1.iloc[0,6]
    else:
        alpha = summary1.iloc[0,7]
        
    return alpha    


In [12]:
#create a function which creates a vector of betas depending on category
def get_beta(column):
    if column == "4_1988":
        beta=summary1.iloc[1,0]
    elif column == "5_1988":
        beta=summary1.iloc[1,1]
    elif column == "6_1988":
        beta=summary1.iloc[1,2]
    elif column == "3_1988":
        beta=summary1.iloc[1,3]      
    elif column == "4_1989":
        beta=summary1.iloc[1,4]
    elif column == "5_1989":
        beta=summary1.iloc[1,5]
    elif column == "6_1989":
        beta=summary1.iloc[1,6]
    else:
        beta = summary1.iloc[1,7]
        
    return beta    
 

### <span style="color:blue"> Now append a colum of $\hat{D}$ to the original dataset and regress $\hat{D}$ on lnw </span>

In [13]:
#create an technical column in the dataset which appends category to year ( represents cartesian product) 
#- necessary to run the function we created

df["cartesian"]= df.lotcateg.astype(str) + "_" + df.year.astype(str)

In [14]:
#apply functions and obtain d_hat
alpha = df.cartesian.apply(get_alpha)
beta =df.cartesian.apply(get_beta)
df["d_hat"]=alpha+beta*df["z"]


In [15]:
#see if it worked 
df

,female,year,lotcateg,z,d,lnw,cartesian,d_hat
0,0,1988,4,0,1,2.751068,4_1988,0.560000
1,1,1989,5,1,1,4.365279,5_1989,0.937063
...,...,...,...,...,...,...,...,...
1474,1,1989,5,1,1,2.857421,5_1989,0.937063
1475,0,1989,6,0,0,2.603315,6_1989,0.257732


In [16]:
#now regress d_hat on lnw using the same loop
#create a new dataframe where will sum the results
summary2 =pd.DataFrame(index=["Beta of d_hat","Casual Effect", "t-stat of Beta", "Category", "Year"])

#iterate/#loop over the list, run regression, extract beta value for Z and t-statistic, append to the dataframe 
for year in years: 
    for cat in categs:
        endog = df["lnw"][df["lotcateg"]==cat][df["year"]==year]
        exog = sm.add_constant(df["d_hat"][df["lotcateg"]==cat][df["year"]==year])

        #fit model for every cat
        mod = sm.OLS(endog, exog).fit()

        #extract params
        d_hat =mod.params.d_hat
        cas = np.exp(d_hat)
        t_stat = mod.tvalues.d_hat

        #shove them all into one list
        add = [d_hat,cas,t_stat, str(cat), str(year)]

        #get new column name in dataset to append the result
        col = "lotcateg_" + str(cat) + str(year)
 
        #now transpose and add the list as a column
        summary2[col]= add
summary2 

,lotcateg_41988,lotcateg_51988,lotcateg_61988,lotcateg_31988,lotcateg_41989,lotcateg_51989,lotcateg_61989,lotcateg_31989
Beta of d_hat,0.119505,0.159243,0.209577,-0.803662,0.0119385,0.518693,0.145346,1.56419
Casual Effect,1.12694,1.17262,1.23316,0.447686,1.01201,1.67983,1.15644,4.77881
t-stat of Beta,0.534346,1.50841,2.28983,-0.761624,0.0406398,2.51007,1.56369,2.1582
Category,4,5,6,3,4,5,6,3
Year,1988,1988,1988,1988,1989,1989,1989,1989


In [17]:
#Compare the resumt with the regression on the instrument directly - the same at in #2 at the end. 
summary

,lotcateg_4_1988,lotcateg_5_1988,lotcateg_6_1988,lotcateg_3_1988,lotcateg_4_1989,lotcateg_5_1989,lotcateg_6_1989,lotcateg_3_1989
Instrument Beta,0.119505,0.159243,0.209577,-0.803662,0.0119385,0.518693,0.145346,1.56419
Casual Effect,1.12694,1.17262,1.23316,0.447686,1.01201,1.67983,1.15644,4.77881
t-stat of Beta,0.546878,1.60704,2.35277,-0.683861,0.0505322,2.34764,1.60404,1.55242
Category,4,5,6,3,4,5,6,3
Year,1988,1988,1988,1988,1989,1989,1989,1989


## <span style="color:red"> We happily conclude that we have arrived at the same results - the top rows are the same </span>

# 3. Verify that the instrumental-variable estimator is equivalent to the quotient of the reduced form to the ﬁrst stage (this quotient is referred to as the Wald estimator). Interpret.

In essense, we need to show that, for each category, $\rho = \frac{E[lnw|z=1]-E[lnw|z=0]}{E[d|z=1]-E[d|z=0]}$

In [18]:
summary3 =pd.DataFrame(index=["Wald_Estimator","Category", "Year"])

#iterate/#loop over the list, get Wald Estimator for each cluster/category 

for year in years: 
    for cat in categs:
        #get the subset of the entire dataset
        slice_df =  df[df["lotcateg"]==cat][df["year"]==year]

        #obtain values to calculate numerator of quotent
        numerator  = slice_df[["lnw","z"]].groupby("z").mean()

        #obtain values to calculate the denominatgor of quotent
        denominator =  slice_df[["d","z"]].groupby("z").mean()

        #calculate the rho / Wald Estimator 
        rho = (numerator.iloc[1,0]- numerator.iloc[0,0])/(denominator.iloc[1,0]- denominator.iloc[0,0])

        #shove them all into one list
        add = [rho, str(cat), str(year)]

        #get new column name in dataset to append the result
        col = "lotcateg_" + str(cat) + str(year)

        #now transpose and add the list as a column
        summary3[col]= add
summary3

,lotcateg_41988,lotcateg_51988,lotcateg_61988,lotcateg_31988,lotcateg_41989,lotcateg_51989,lotcateg_61989,lotcateg_31989
Wald_Estimator,0.119505,0.159243,0.209577,-0.803662,0.0119385,0.518693,0.145346,1.56419
Category,4,5,6,3,4,5,6,3
Year,1988,1988,1988,1988,1989,1989,1989,1989


In [19]:
#compare the estimatores with the summary in the #2 of the problem
summary

,lotcateg_4_1988,lotcateg_5_1988,lotcateg_6_1988,lotcateg_3_1988,lotcateg_4_1989,lotcateg_5_1989,lotcateg_6_1989,lotcateg_3_1989
Instrument Beta,0.119505,0.159243,0.209577,-0.803662,0.0119385,0.518693,0.145346,1.56419
Casual Effect,1.12694,1.17262,1.23316,0.447686,1.01201,1.67983,1.15644,4.77881
t-stat of Beta,0.546878,1.60704,2.35277,-0.683861,0.0505322,2.34764,1.60404,1.55242
Category,4,5,6,3,4,5,6,3
Year,1988,1988,1988,1988,1989,1989,1989,1989


## <span style="color:red"> We again happily conclude that the results match -  the top two rows are the same for each cluster </span>